In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%autoreload

import h2o

from utilities import create_data as create
from utilities  import create_delivery_file as delivery
from utilities import save_model as save
from h2o.automl import H2OAutoML

use_mean_values = True
model_name = '27E_h2o_stacked_ensemble_mean_no_bad_features'

In [12]:
data = create.CreateTrainingData()
train = data.create_training_data(use_mean_values)
test = create.create_test_data(use_mean_values)
target = create.create_target_data()

In [13]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/stinky/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpzp6v8uy2
  JVM stdout: /tmp/tmpzp6v8uy2/h2o_stinky_started_from_python.out
  JVM stderr: /tmp/tmpzp6v8uy2/h2o_stinky_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_stinky_5c5plt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.889 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [ ]:
# Delete all rows with nans in pv_measurement column in train
# train = train[train['pv_measurement'].isna() == False]

X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

In [ ]:
autoML = H2OAutoML(max_models=30, seed=4, nfolds=5)
autoML.train(x=X, y=y, training_frame=X_train)

In [ ]:
best_model = autoML.leader
save.save_model(best_model, model_name)

In [ ]:
# Remove features that are not in the training set
# test.fillna(0, inplace=True)
test_pd = test[X]
test_h2o = h2o.H2OFrame(test_pd)

pred = autoML.predict(test_h2o)

In [ ]:
file = delivery.CreateDelivery(pred, 'mean_features_in_training_no_feature_engineering')
file.create_delivery_file(use_mean_values)